# Решение ДЗ по Дискретной математике

Для начала необходимо переопределить свой граф, а также начальную и конечную вершину в функции `get_graph`.
Граф задаётся списком смежности -- каждой вершине в словаре соответствует словарь вида "`смежная вершина`: `максимальный поток`".
Вершинами может быть любой хэшируемый объект питона (числа, строки).

In [3]:
def get_graph():
    """Define your graph here"""
    vertex_number = 12
    start_vertex, target_vertex = 1, vertex_number
    graph_table = [
        #  x1,  x2,  x3,  x4,  x5,  x6,  x7,  x8,  x9, x10, x11, x12

        "  -    9    3    20   41   -    -    -    -    -    -    - ",  #   x1
        "  -    -    5    -    -    -    -    -    -    -    -    10",  #   x2
        "  -    -    -    29   -    5    2    -    -    -    -    - ",  #   x3
        "  -    -    -    -    -    14   -    22   -    -    -    - ",  #   x4
        "  -    -    -    27   -    -    -    15   -    18   -    - ",  #   x5
        "  -    6    -    -    -    -    13   -    -    -    -    33",  #   x6
        "  -    -    -    -    -    -    -    -    32   -    20   - ",  #   x7
        "  -    -    -    -    -    -    23   -    11   2    -    - ",  #   x8
        "  -    -    -    -    8    -    -    -    -    -    1    10",  #   x9
        "  -    -    -    -    -    -    -    -    -    -    -    11",  #  x10
        "  -    -    -    -    -    -    -    -    -    14   -    12",  #  x11
        "  -    -    -    -    -    -    -    -    -    -    -    - ",  #  x12
    ]

    # graph[from_vertex_i] = {to_vertex_j: max_flow_j, ...}
    graph_dict = {i+1: {} for i in range(vertex_number)}
    for i, line in enumerate(graph_table):
        for j, value in enumerate(line.split()):
            if value != '-':
                graph_dict[i+1][j+1] = int(value)

    return Graph(graph_dict, start_vertex, target_vertex)

После определения своего графа, просто запустить все ячейки и смотреть вывод в самом низу

In [ ]:
def dfs_paths(graph, start, finish):
    paths = []
    visited = {k: False for k in graph.keys()}
    current_path = []
    _dfs_paths(graph, start, finish, visited, paths, current_path)
    return paths


def _dfs_paths(graph, start, finish, visited=None, paths=None, current_path=None):
    visited[start] = True
    current_path.append(start)

    if start == finish:
        paths.append(tuple(current_path))
    else:
        for v in graph[start].keys():
            if not visited[v]:
                _dfs_paths(graph, v, finish, visited, paths, current_path)

    current_path.pop(-1)
    visited[start] = False


def dfs_incremental_routes(graph):
    paths = []
    visited = {k: False for k in graph.flows.keys()}
    current_path = []
    _dfs_incremental_routes(graph, graph.start_vertex, visited, paths, current_path, '+')
    return paths


def _dfs_incremental_routes(graph, current_vertex, visited=None, paths=None, current_path=None, from_vertex=''):
    visited[current_vertex] = True
    current_path.append((current_vertex, from_vertex))

    if current_vertex == graph.target_vertex:
        paths.append(tuple(current_path))
    else:
        for j in graph[current_vertex].keys():
            if not visited[j]:
                f = graph[current_vertex][j]
                if f.max_value != f.value:
                    _dfs_incremental_routes(graph, j, visited, paths, current_path, f"+{current_vertex}")
        for v in graph.get_reversed()[current_vertex].keys():
            if not visited[v]:
                f = graph[v][current_vertex]
                if f.value > 0:
                    _dfs_incremental_routes(graph, v, visited, paths, current_path, f"-{current_vertex}")

    current_path.pop(-1)
    visited[current_vertex] = False


def dfs_mincut_search(graph):
    visited = {k: False for k in graph.flows.keys()}

    result = []
    queue = [(graph.start_vertex, '+')]

    while queue:
        current_vertex, mark = queue.pop(0)
        result.append((current_vertex, mark))
        visited[current_vertex] = True

        for j in graph[current_vertex].keys():
            if not visited[j]:
                f = graph[current_vertex][j]
                if f.max_value != f.value:
                    queue.append((j, f"+{current_vertex}"))
        for v in graph.get_reversed()[current_vertex].keys():
            if not visited[v]:
                f = graph[v][current_vertex]
                if f.value > 0:
                    queue.append((v, f"-{current_vertex}"))

    return result

from zlib import compress
import base64
import string

URL = 'https://www.plantuml.com/plantuml/png/'

plantuml_alphabet = string.digits + string.ascii_uppercase + string.ascii_lowercase + '-_'
base64_alphabet = string.ascii_uppercase + string.ascii_lowercase + string.digits + '+/'
b64_to_plantuml = bytes.maketrans(base64_alphabet.encode('utf-8'), plantuml_alphabet.encode('utf-8'))


class Output:
    def __init__(self):
        self.markdown_text = ''
        # self.text("""<style type="text/css"> img {height: 250px;}</style>""")

    def text(self, markdown_text, end='\n'):
        self.markdown_text += markdown_text + end

    def math(self, math_text, end='\n'):
        self.markdown_text += f"$${math_text}$$" + end

    def image(self, url, text='', end='\n'):
        self.markdown_text += f"![{text}]({url})" + end

    def get_text(self):
        return self.markdown_text

    def print(self):
        try:
            from IPython.display import Markdown as print_markdown
        except ImportError:
            print_markdown = print
        print_markdown(self.markdown_text)


def deflate_and_encode(plantuml_text):
    """zlib compress the plantuml text and encode it for the plantuml server."""
    zlibbed_str = compress(plantuml_text.encode('utf-8'))
    compressed_string = zlibbed_str[2:-4]
    return base64.b64encode(compressed_string).translate(b64_to_plantuml).decode('utf-8')


def build_graph(graph, marked_vertex=None, footer=""):
    if marked_vertex is None:
        marked_vertex = {}

    definition = [f'() "<latex>x_{{{i}}}</latex>" as x{i}' if i not in marked_vertex else f'() "<latex>x_{i} ({marked_vertex[i]})</latex>" as x{i} #pink;line:red;line.bold;text:red' for i in sorted(graph.keys())]

    x = []
    for start, xxx in graph.items():
        for to, flow in xxx.items():
            if flow.value < flow.max_value:
                x.append(f"x{start} --> x{to} : {flow.value} / {flow.max_value}")
            else:
                x.append(f"x{start} --> x{to} #line:blue;line.bold;text:blue : {flow.value} / {flow.max_value}")

    return "@startuml\n"\
           "left to right direction\n"\
           f'footer "{footer}"\n' + \
           '\n'.join(definition) + '\n' + \
           '\n'.join(x) + \
           '\n@enduml'


def get_url(plantuml_text, url=URL):
    return url + deflate_and_encode(plantuml_text)

from collections import namedtuple
from random import shuffle

Flow = namedtuple('Flow', ['value', 'max_value'])


def walk(path):
    p0 = path[0]
    for i in range(1, len(path)):
        yield p0, path[i]
        p0 = path[i]


class Graph:
    def __init__(self, graph_dict, start_vertex, target_vertex):
        """
        :param graph_dict: graph_dict[from_vertex][to_vertex] = max_flow
        """

        self.start_vertex = start_vertex
        self.target_vertex = target_vertex

        self.flows = {}
        for from_vertex in graph_dict.keys():
            self.flows[from_vertex] = {}
            for to_vertex in graph_dict.get(from_vertex).keys():
                max_flow = graph_dict[from_vertex][to_vertex]
                self.flows[from_vertex][to_vertex] = Flow(0, max_flow)

        self.paths = dfs_paths(self.flows, self.start_vertex, self.target_vertex)
        shuffle(self.paths)

    def __getitem__(self, item):
        return self.flows.__getitem__(item)

    def get_flow(self, from_vertex, to_vertex):
        return self.flows[from_vertex][to_vertex]

    def increment_flow(self, from_vertex, to_vertex, by_value):
        f = self.get_flow(from_vertex, to_vertex)
        self.flows[from_vertex][to_vertex] = Flow(f.value + by_value, f.max_value)

    def is_saturated_flow(self, from_vertex, to_vertex):
        f = self.flows[from_vertex][to_vertex]
        assert f.value <= f.max_value
        return f.value == f.max_value

    def get_reversed(self):
        reversed_graph = {key: {} for key in self.flows.keys()}
        for i in self.flows.keys():
            for j in self.flows[i].keys():
                reversed_graph[j][i] = self.flows[i][j]
        return reversed_graph

    def get_incremental_routes(self):
        return dfs_incremental_routes(self)

    def get_mincut(self):
        mincut = dfs_mincut_search(self)
        flow = 0
        X1 = {v for v, x in mincut}
        X2 = set(self.flows.keys()) - X1
        for v in X1:
            for j in self.flows[v].keys():
                if j in X2:
                    flow += self.flows[v][j].value
        return mincut, flow

    def current_state_image(self, marked=None):
        return get_url(build_graph(self.flows, marked))

from random import choice


class Solver:
    def __init__(self, graph):
        self.graph = graph
        self.out = Output()
        self.flow = 0

    def run(self):
        self.stage1()
        self.stage2()
        # print(self.out.markdown_text)

    def stage1(self):
        self.out.text(r"1. $\varphi = 0$")
        self.out.text(r"2. $\varphi_п - ?$")

        _step = 0
        for path in self.graph.paths:
            delta = []
            for from_vertex, to_vertex in walk(path):
                f = self.graph.get_flow(from_vertex, to_vertex)
                delta.append((f.max_value, f.value))

            min_delta = min([m - v for m, v in delta])
            if min_delta > 0:
                _step += 1; __path = ','.join([f'x_{{{p}}}' for p in path])
                __delta = ','.join([f'{m}-{v}' for m, v in delta])

                _sat = []
                for from_vertex, to_vertex in walk(path):
                    self.graph.increment_flow(from_vertex, to_vertex, min_delta)
                    _sat.append((from_vertex, to_vertex))
                self.flow += min_delta

                __sat = ','.join([f"(x_{{{s}}}, x_{{{e}}})" for s, e in _sat if self.graph.is_saturated_flow(s, e)])
                self.out.text(fr"    {_step}. $({__path})$", end='\n'*2)
                self.out.text(fr"       $\delta^* = min\{{{__delta}\}} = {min_delta}$", end='\n'*2)
                self.out.text( r"       $\Rightarrow \text{ребра } " + __sat + r"\text{ - насыщенные}$", end='\n'*2)
                self.out.text(fr"       $\varphi = {self.flow}$")
                self.out.text(  "       ", end=''); self.out.image(self.graph.current_state_image())

        self.out.text("       ---\n"*2)
        self.out.text(r"       $\varphi_п =" + f"{self.flow}$")
        self.out.text("       ", end=''); self.out.image(self.graph.current_state_image())
        self.out.text("       ---\n"*2)

    def stage2(self):
        self.out.text(fr"3. $\varphi_{{\max}}$ -- ?", end='\n'*2)

        _step = 0
        while True:
            incremental_routes = self.graph.get_incremental_routes()
            if not incremental_routes:
                break

            incremental_route = choice(incremental_routes)
            edges = [(int(s[1:]), end, s[0]) for end, s in incremental_route[1:]]

            delta, phi = [], []
            for from_vertex, to_vertex, direction in edges:
                if direction == '+':
                    f = self.graph.get_flow(from_vertex, to_vertex)
                    delta.append((f.max_value, f.value))
                else:
                    f = self.graph.get_flow(to_vertex, from_vertex)
                    phi.append(f.value)

            if delta and phi:
                __delta = ','.join([f'{m}-{v}' for m, v in delta]); __phi = ','.join([f'{v}' for v in phi])

                delta = [m - v for m, v in delta]
                eps = min([min(delta), min(phi)])

                if eps > 0:
                    _step += 1; __path = ','.join([f'x_{{{p}}}({x})' for p, x in incremental_route])
                    __forward_edges = ','.join([f"(x_{{{from_vertex}}},x_{{{to_vertex}}})" for from_vertex, to_vertex, direction in edges if direction == '+'])
                    __backward_edges = ','.join([f"(x_{{{from_vertex}}},x_{{{to_vertex}}})" for from_vertex, to_vertex, direction in edges if direction == '-'])

                    for from_vertex, to_vertex, direction in edges:
                        if direction == '+':
                            self.graph.increment_flow(from_vertex, to_vertex, eps)
                        else:
                            self.graph.increment_flow(to_vertex, from_vertex, -eps)
                    self.flow += eps

                    self.out.text(f"    {_step}. Удалось пометить: $({__path})$", end='\n'*2)
                    self.out.text(fr"       Прямые рёбра: ${__forward_edges}$", end='\n'*2)
                    self.out.text(fr"       Обратные рёбра: ${__backward_edges}$", end='\n'*2)
                    self.out.text(fr"       $\delta^* = min\{{{__delta}\}} = {min(delta)}$", end='\n'*2)
                    self.out.text(fr"       $\varphi^* = min\{{{__phi}\}} = {min(phi)}$", end='\n'*2)
                    self.out.text(fr"       $\varepsilon^* = min\{{{min(delta)}, {min(phi)}\}} = {eps}$", end='\n'*2)
                    self.out.text(fr"       $\varphi={self.flow}$", end='\n'*2)
                    self.out.text("       ", end=''); self.out.image(self.graph.current_state_image(dict(incremental_route)))

        self.out.text("---")

        if _step == 0:
            self.out.text("# Похоже, не нашлось увеличивающих маршрутов. Попробуйте перезапустить, чтобы найти другое случайное решение")
            return

        mincut, mincut_flow = self.graph.get_mincut()
        if mincut_flow != self.flow:
            self.out.text("# Похоже, пропускная способность минимального разреза не ровна максимальному потоку. Если это случается, значит что-то явно пошло не так.")
            return

        self.out.text(f"## Минимальный разрез (выделенные вершины):")
        self.out.image(self.graph.current_state_image(dict(mincut)))

        self.out.text("---")
        self.out.text(f"Достигнут полный поток за {_step+1} итераций.")
        self.out.text(f"Нашли {_step} увеличивающих маршрутов.")
        self.out.text(f"...")
        self.out.text(f"Подтверждается теорема Форда-Фалкерсона ...")
        self.out.text("\n" + fr"$\varphi_{{max}} = {self.flow}$")

In [4]:
solver = Solver(get_graph())
solver.run()

try:
    from IPython.display import Markdown as print_markdown
except ImportError:
    # Если каким-то образом запускать не через Jupyter, то для просмотра следует
    # вставить весь сырой вывод в любой нормальный markdown-редактор (например, https://stackedit.io/)
    print_markdown = print
print_markdown(solver.out.markdown_text)

59


<style type="text/css"> img {height: 250px;}</style>
1. $\varphi = 0$
2. $\varphi_п - ?$
    1. $(x_{1},x_{5},x_{4},x_{6},x_{2},x_{12})$

       $\delta^* = min\{41-0,27-0,14-0,6-0,10-0\} = 6$

       $\Rightarrow \text{ребра } (x_{6}, x_{2})\text{ - насыщенные}$

       $\varphi = 6$
       ![](https://www.plantuml.com/plantuml/png/NPFBReCm44Nt_WgBRjf56iOFC4aKvKygMfmKYO94N8dFxz3OGyOxTCxrbNEjNEwXdSFlpo06VmqoJ7BkRzz1Tltilq8_ZU8wJS7FiYZ4wviiJaCR_79UFk2qVtmMihtB1RXLt2fkDRUQMyEjuTPoQxcjkAsuhRcjkNNSEcuRRfjiZJAReynyFbUs5sI3WH8BoDtkZDl9WopbNZP4T2IQY8b4bOGieWgH0O4Lx9mb0YgcyBweULT3gDeEHbAdaC0yPVwR326NGfYoM_cQKZMXb09B2Ch4dC3YECFBq8_-CFlkk7wyVqv3TyJ5mefYRKNnT47GX20jgP7L3zOaf0W1F8tfif-h2QMJ-989qRMnhDbUOIME2DN3XjB5rbV0aWnY420VnZn1IYfnyME7V-O_)
    2. $(x_{1},x_{4},x_{8},x_{7},x_{9},x_{5},x_{10},x_{12})$

       $\delta^* = min\{20-0,22-0,23-0,32-0,8-0,18-0,11-0\} = 8$

       $\Rightarrow \text{ребра } (x_{9}, x_{5})\text{ - насыщенные}$

       $\varphi = 14$
       ![](https://www.plantuml.com/plantuml/png/XPFDRiCW48JlF0LoB-qXZHVy0qaKvKsgj2Qf9TUM72hvyRiuiDPowStwPfXT1ldoyDVP__uCOd0tB_qav_x-xMNNp-xBzzCeRjFatIoBGho-oU8qNBrRpii7dFRFpq9U7t81hYgkAgvghcgkLbojk5fpjUPgmzM6gorNMwuQhXgkMgxQh8qogwFCzBokh2_82WCb5f2xtHcxamTPohsqH7GacaY5n21H9Q4QKOEe0e4Hx5nD15HquRueKrPmAKkesGv6qYQJG3zvrgIAa4bReQlUmeDBjOHCZ8AQ49Jn91Y1mR66bw4VtM5stJ5yl7zEGtV4ndn0CROXUreGD258JMfaxPFPE4WhGW3fRjY9sTBMcsX2DWq0GklQiHMRNk6VhKEg8JlDXWsbhSCJuOGDcZWT81iVde0W7LVYuiOE__G_)
    3. $(x_{1},x_{5},x_{8},x_{7},x_{9},x_{12})$

       $\delta^* = min\{41-6,15-0,23-8,32-8,10-0\} = 10$

       $\Rightarrow \text{ребра } (x_{9}, x_{12})\text{ - насыщенные}$

       $\varphi = 24$
       ![](https://www.plantuml.com/plantuml/png/XPJDReCm48JlVWgBNjf36jRcnqwYA6zIfSL9aIX8n9Lu_Au1BrrVSeDlnkFr675v-EleVtywqRcRbtwGOtl_zh9fH_Vbsw4Njs7mRfHP9bxVP7RghjvDv-a3JllbCPFNXvo0gughYgkQgvgh1LSBhfPSBRbQSRNYQitLcgk6guQhbgisQIDFwiWJFQqhwGkIma292UHkTyRkv47cSYyj4RqIJQH0Of2ed52922fa1GZCO0jB8X3IWmi7Hfd2WajPGjMsS2LrD0dqasTEAWYPE1QwoYqyk5HDABWWP4593D-NfM049gz5l7HjxmwZQuxXuVrpw9eZTkO3MdChiiS9GHE2sAL6LY_CXjt3JbeH0uYdmrRC5ZVxb2PcunP0Q1vyBSR6ctWoTqWrPAVDOKDFZxxq5su__tTKiyu6a0mNxeXWB0M_kOlh6_mN_04=)
    4. $(x_{1},x_{2},x_{3},x_{6},x_{7},x_{11},x_{10},x_{12})$

       $\delta^* = min\{9-0,5-0,5-0,13-0,20-0,14-0,11-8\} = 3$

       $\Rightarrow \text{ребра } (x_{10}, x_{12})\text{ - насыщенные}$

       $\varphi = 27$
       ![](https://www.plantuml.com/plantuml/png/ZPJDReCm48JlVWgBNjf36jRcnqwYA6zIfSLfaIX8n9Lu_AwDMRfSqXjyCpjUnb4kTtyT_Szt9pftyz8FScm_lxnisj5z-7RenMqOl1jbbedd5vcTkgjtqtbwWzD-VipazIuduAhYgkAgvghcQi7LWgibLqkkLbojk5fpjUQguQhXgkMgtRIHR-h8D_gsharVi2aCb9X0xdPdx4uUf9PxQObe93aIJQH0Of2ed52922fa1GZCeC4GLHA1a1vSk33A51RYbILKhOE9r8j9e9yyCQaWP9Qrq5MkuS6bQaB11I4BIcBuFe-24PYSYdZgsjuTHjSSmyFh-z0rH-pC1vHoAxBNQMVGXEBNnkQGrJEpuVHmabR40D9iwDEiST6dD36Rul0s5nGNJyNOvIOUx1rI3TdZuJ5rHGy_VUult7x-xn58cmFyMLsd6oIuL8Q_o8lh6_od-0K=)
    5. $(x_{1},x_{2},x_{3},x_{4},x_{8},x_{9},x_{11},x_{12})$

       $\delta^* = min\{9-3,5-3,29-0,22-8,11-0,1-0,12-0\} = 1$

       $\Rightarrow \text{ребра } (x_{9}, x_{11})\text{ - насыщенные}$

       $\varphi = 28$
       ![](https://www.plantuml.com/plantuml/png/ZPJDReCm48JlVWgBNjf36jRcpqaKvKsgj3Wj4WM9k1AFtrqmIvTBUeDlnkFr676vX-iGVhvRrVfRqA7NG_Fv5NJT3FuZD7sdRdqV_A2JH3s_wEJKNeCVp-CRdFRpOwAlTps2L8rKZLIjLArKCwbcKisbcakra6eXrLAgfLGhgLPITL9rcpRIJHtfHj_MjUaBDeM1KIFetUwCtUc3plHUEoOMIOh4CicGL4XCoYX710Mo31Hcy4AAofa0fPCB1qQPmu327AC2aPqNHbB66N3ERFLCIHcZ2f4Z4xhoDPnSfcH4Bg0io9dX-tmYg1GcnoAUsgRpXy7NHtfuVU_R-eYT1K8njs1x6MS6osWwxTGSid9cZdQddQnX1X3NKf_L6ZVvZ6NcObDucmkQ1e_5kEKc7inDgHNRSVC7_cdBrV-mbxbS-ZJIVozPZWNmvrmsNYz3iqnZrCLtDVv6VW4=)
    6. $(x_{1},x_{2},x_{3},x_{7},x_{11},x_{12})$

       $\delta^* = min\{9-4,5-4,2-0,20-3,12-1\} = 1$

       $\Rightarrow \text{ребра } (x_{2}, x_{3})\text{ - насыщенные}$

       $\varphi = 29$
       ![](https://www.plantuml.com/plantuml/png/ZPJTReCm38NlynGHkzakkkA4dwIjghx9r0tQ8R4YqKpYyMT3S6jks1ryvyIEZn6dUpZtuVUdLMrz2Jfqkc-ktq5NJLz_XQQxgKlNXRhNIQ9UttHoQC-X7ex31nosqsEYptSzW5IDL8rKhLIjL3EfPbBDfPfBjP1g8TLIggLKdLITLBrK_IADT157kj2NSItoWaLWODG0UhCvOdPwftEzrPw9HP8YiKmo90w9IHdbYA10be726l7WIzlSwbrVLtjwUF_iscgFNGEXs2TdEr1hAe7Ju5dk1DJ9CoeGsUbW9ELiKkXdprWfO-GGUJAXAtyK9vSf6P4BQ5Z8cU7xD2uuXPLZIYkZKTs2xMMyCrX6uxHZhCZAYNdgJfsiOGQ0C8jXkwU1nmLOPZw6WAkUqNZnT0h6psjQkJTLTMp7vYl-iUN3lvhB52vzD-c_ZynZ0JpDPUDw6LAUXg1Hf_fMuJ_c3m==)
    7. $(x_{1},x_{5},x_{4},x_{6},x_{12})$

       $\delta^* = min\{41-16,27-6,14-6,33-0\} = 8$

       $\Rightarrow \text{ребра } (x_{4}, x_{6})\text{ - насыщенные}$

       $\varphi = 37$
       ![](https://www.plantuml.com/plantuml/png/ZPJTReCm38NlynGHkzakkk8a_AIjghx9n0RTa5YHQ2Rn-BCXU3Ktx0w-O_l49uZBFLH3-FdkLDTSWmwz7jhFhw3hTcW-GjlVrBNlGpFe957FBpevTLLenlFu1gVz_9Zewgv7aAgHgf6gbQgLgfEgasecrKogkLHpgHPIBQHQIhMKgfUgNwMHhk98L_ewhbLUi0eCZ1f1xtPdp4uVTARtsZEnI58abeb3KY8nAQCCaN7872YS4HkVklRM78QcFjB3wtlVrKTq3OIYJyRbGDOv8jm6Uza9oCapoX7PkJ6IOYbIM1zhDfo1ZWgE6qfadaRWZEp1cifCmOpAWBA0Z1c-psb0gT0uXhZXJslcN5x4bS0ociAWgHPPCJDFxXCpp00GkdWNvKCUq_rOPZvcX5_2WgQ3ftDkVhd5ZNFJr9BBqNmhPxBygz_CPGwNFglqtotBMW0FUzbulmmfJqFGg4jpg_4Nz0i=)
    8. $(x_{1},x_{3},x_{4},x_{8},x_{7},x_{11},x_{12})$

       $\delta^* = min\{3-0,29-1,22-9,23-18,20-4,12-2\} = 3$

       $\Rightarrow \text{ребра } (x_{1}, x_{3})\text{ - насыщенные}$

       $\varphi = 40$
       ![](https://www.plantuml.com/plantuml/png/ZPJ1ReCm38RlUOg8NRP3LvmG86rLzKscRj0EYHM9PXAFFnk2EtEXD_Z-FxRp9t2wXtCVVdzQrTQNe4Ed--Rw7NJLzFLNQBgRkdHTg7kT9EhrJIU7zXpguJXym64xFIRwVDS3IDL8rKZLIjLADPDg9bKdLIVLNAgvL0kf5b8jfLfArKlLBz985t6a2tqPro8lM0G6HWsWDviZPgTtskcjzaoiVccRMxthwsfF3--VNLljSN4WX7QBTils34c9nAIC72AJ8Sj0OOFdgj8GZkr0S-M8SAkubZjHLUCPvN4SnwH00cHIw8-UbSv0HI7Z1IKnGpMmY7lq7dscO3RwKbhhcE7x50UK2Zl7Y5S6e7tcR2_Ydi0o6jEWgXPPCJ5FtKTcc04WBE9Xb8zoPap4ClCn9BmdCne7JwVW_7oCAtDJzvBjs7mjQ6hvyA_cCeLBzofzUicyBO1_-xBnW1dEjnu_X5Dzg_07zGS=)
    9. $(x_{1},x_{5},x_{8},x_{7},x_{11},x_{12})$

       $\delta^* = min\{41-24,15-10,23-21,20-7,12-5\} = 2$

       $\Rightarrow \text{ребра } (x_{8}, x_{7})\text{ - насыщенные}$

       $\varphi = 42$
       ![](https://www.plantuml.com/plantuml/png/ZPHDReCm48NtFeL5D-qY3MDZZ9CeoasgjD0MYGQ9k1B7xmmnamuRihE-zpm_puZJDPw7-FlJgQxvZ3hsUcY_lgEksw7vY6r_KPzz7vj1PvbwVj7PeJl7PZoERt3OtewPFb_r258rKZLIjLArKYsaMaZLITL9jPHgALKlLI_LIggLL8DKmoADV157lj2NSItoWaLWODG8UhCvOdPwfvtUwi34wgUklJIxeQdtT7XzxxjwZvSZ8RHRj5ks5qWg92Pdv12P4ba12XiyLfM6S6m7cejAuAfubpiLr2amAjCuZeb70cHIw4-UbSv0HQ7W2nKnGpMmYBltddp6CsCVE6QGfpYWKjWvHRmo0ErPijsddS0ocjAWgXQPlx50tITcc04W3EanghdSIdT3pOpb2o4bY1_HZAQjybkgONxZbR8qMiLsd6pj5QZbtRyQ6kJfwy33erVcjG3-xMNJwpEai3r1eqxDfSQ_rny=)
    10. $(x_{1},x_{2},x_{12})$

       $\delta^* = min\{9-5,10-6\} = 4$

       $\Rightarrow \text{ребра } (x_{1}, x_{2}),(x_{2}, x_{12})\text{ - насыщенные}$

       $\varphi = 46$
       ![](https://www.plantuml.com/plantuml/png/ZPJ1ReCm38RlUOg8NRP3LvmGIDggwfjCtQ0T4YiIpIGUVtO1T-O2D_JzVspdTyJf7izT_FrfL5DTeewjxkhhTzHbtLLViMvlwjAsiUfqagZNDvqScdEi-cF_0OVjy9deyrtt85KZLIDLArKhrKogcLITL9rKSwdcKYsaMaZLIzLBDKWrpD98Pt6aCtqUromlc0K6HlMWDviZPgTVclfMxRggtDF7-sVRb7kqHa9wfuFUwi1sksItQBTipv1u92Pbv12P75a62hklg-hGxzWEYqD3YWSm9QpCSsGqHs2KZyCw9WKT99D2_-XPwaD58UC3dfYX6bZ4FNi_VAPWnZvmp73c8Ipm2Zklsmu5cREz6Ey4bX5s6QfQPCN00dL_CCCCODmrhSfFvHQw6sfcB1y8OuBu_YRqk5Kwf1gc5x1GbaRpRCV9bhP0BP_-zQy3FzOUcQu5yEzUTjm-Gmgx86ZKgRgL-EFx0m==)
       ---
       ---

       $\varphi_п =46$
![](https://www.plantuml.com/plantuml/png/ZPJ1ReCm38RlUOg8NRP3LvmGIDggwfjCtQ0T4YiIpIGUVtO1T-O2D_JzVspdTyJf7izT_FrfL5DTeewjxkhhTzHbtLLViMvlwjAsiUfqagZNDvqScdEi-cF_0OVjy9deyrtt85KZLIDLArKhrKogcLITL9rKSwdcKYsaMaZLIzLBDKWrpD98Pt6aCtqUromlc0K6HlMWDviZPgTVclfMxRggtDF7-sVRb7kqHa9wfuFUwi1sksItQBTipv1u92Pbv12P75a62hklg-hGxzWEYqD3YWSm9QpCSsGqHs2KZyCw9WKT99D2_-XPwaD58UC3dfYX6bZ4FNi_VAPWnZvmp73c8Ipm2Zklsmu5cREz6Ey4bX5s6QfQPCN00dL_CCCCODmrhSfFvHQw6sfcB1y8OuBu_YRqk5Kwf1gc5x1GbaRpRCV9bhP0BP_-zQy3FzOUcQu5yEzUTjm-Gmgx86ZKgRgL-EFx0m==)
       ---
       ---

3. $\varphi_{\max}$ -- ?

    1. Удалось пометить: $(x_{1}(+),x_{5}(+1),x_{4}(+5),x_{3}(-4),x_{6}(+3),x_{7}(+6),x_{11}(+7),x_{12}(+11))$

       Прямые рёбра: $(x_{1},x_{5}),(x_{5},x_{4}),(x_{3},x_{6}),(x_{6},x_{7}),(x_{7},x_{11}),(x_{11},x_{12})$

       Обратные рёбра: $(x_{4},x_{3})$

       $\delta^* = min\{41-26,27-14,5-3,13-3,20-9,12-7\} = 2$

       $\varphi^* = min\{4\} = 4$

       $\varepsilon^* = min\{2, 4\} = 2$

       $\varphi=48$

       ![](https://www.plantuml.com/plantuml/png/bPN1ReCm38RlF8L5BwschZWXGDggwfjCtKetD0OJpQG-_coWxiXXm0t-_Fxis4OSh-xKk9-lCYZpYmDNGrEyVpWu5qt-vegw2YvrxV86mZ1OhY3SboUNtmwt5uJbqsg_wLv3E5tXXh3uBghFNLbK-RR9p-t3ysjTddTaSYmD6MeOhuQd6fRhs4kXvwQ8gKpZGUAv44CGz21cBYGXYFOWoLn8If34WwHp8TamFXkUsk6fzSOUUJEFl7FUYzHVZDcRWOfRxlUSr74EhU5wVQ1jWiK_DjY2XGrOiUinkoQx5djCIaQAYaGo92dMOWme-pIg8RyHEuuMZH45KDE9B7KehiEAb4pCsTjJfZ8X8DWq0CPSH2m16Mjj5GGnZy8mOIqLJNneHACBTPt4BA3CqqR7t4JiQNydr29HbewgIKixpNBsLbEYqMQIYFqWipjl9BtYNL1Q0cpVGaIHscj5NLljVJz6i5nR9dR-Q4R6m2aV_kcxGmzJG-xNGlnpBzsFNqJLVqweWcDUdUbtyWi=)
    2. Удалось пометить: $(x_{1}(+),x_{5}(+1),x_{10}(+5),x_{11}(-10),x_{7}(-11),x_{6}(-7),x_{12}(+6))$

       Прямые рёбра: $(x_{1},x_{5}),(x_{5},x_{10}),(x_{6},x_{12})$

       Обратные рёбра: $(x_{10},x_{11}),(x_{11},x_{7}),(x_{7},x_{6})$

       $\delta^* = min\{41-28,18-8,33-8\} = 10$

       $\varphi^* = min\{3,11,5\} = 3$

       $\varepsilon^* = min\{10, 3\} = 3$

       $\varphi=51$

       ![](https://www.plantuml.com/plantuml/png/bLN1ReCm3BtdAo9wwJHrnGaXfAsg_idK3RgXCPXe9lNpPxVKLN8OS47X-VdvEJRYSFR7tlz-DwAfJbxwJlRrnwUNPTrNxxxkMd7gEb_rCad4yaack-Regyl-yWfo-VoqMzzU4tayoml8nK_TVcsRkgqsVLLU3ozlNLDkaUG92ZLKcA_2g0wZEencOJGBemRTGMJFpBMNo-NAHYBvN16B8X1RiNDLYZ2_2ACkZBfeKYbUXebdbSwr042TfB7ExA63GZjvBACcoE22hLPxt1Ev-8ScDzB9jNHCrsDqZNJDz0oH0X6LCcI818Xb8B3wD5M3VCDq63MD4rdZGw0o-r3amp6KJwmvq2sfae90iMa2a969Z1CAmgukKCGyZ45Ec6MCUM0OmuOG90DG22mzRNOad3FT3fS0cY4iCyXg1EqDT8JHrBHY33cFIHPtlP7oYfP1QKvmmnq2C7JjAxtTgxilo8WisIYOJX_Do1oev8C_VNdmC3NbtXP0r1VDdq4rV4-WnA5gI_nJ_04=)
    3. Удалось пометить: $(x_{1}(+),x_{4}(+1),x_{8}(+4),x_{9}(+8),x_{7}(-9),x_{11}(+7),x_{12}(+11))$

       Прямые рёбра: $(x_{1},x_{4}),(x_{4},x_{8}),(x_{8},x_{9}),(x_{7},x_{11}),(x_{11},x_{12})$

       Обратные рёбра: $(x_{9},x_{7})$

       $\delta^* = min\{20-8,22-12,11-1,20-8,12-9\} = 3$

       $\varphi^* = min\{18\} = 18$

       $\varepsilon^* = min\{3, 18\} = 3$

       $\varphi=54$

       ![](https://www.plantuml.com/plantuml/png/bLNBReD03Bpp5HQv9AhIu5qUIn95-PCgBQH5fL2HhPJFhqs8czr3WHkCn-Fn0t6ys5DdVxxge2xF5cmBNVN-OQ6eklBDLcqJdDlMbXs4OR1SGRYlJxQy7gul2CkdrNvpUmtXT84hmkAxQZvtTTMKswui-eVdrxOkTaIo3BaQoirNRbIxKUr6OwgFde5uhe74pK_SQEf6KpUQmNATUmQokGOCTH5x8cQkI4uYnXF9vufWv2qpykAyyCpV-EoLe-Az-OiZT5o7xckzFj2PmE8V6cmXXmtaGjTZT4rqBVIO46fh0oeIB64MOp46L7wQR4ByHEWuwXeZIg2fax8OKUmZ5oYTM7EWPwpA2W69JHF0c4t4ac0OOrL58id3X5B6Ci5w7hl5H31gQ1Wbce1AJriU2wT2puOXe1Q8oWooci3i1kPtdre9Hfo7LPgxu4XznUUWj2ImSZyK5AXlBBeDDhzVo8Yi8ReHEdyt8ulWad_ywTT33rDJS6WBqUkB3q30SPr-GIeubar1Fv9V)
    4. Удалось пометить: $(x_{1}(+),x_{4}(+1),x_{3}(-4),x_{7}(+3),x_{6}(-7),x_{12}(+6))$

       Прямые рёбра: $(x_{1},x_{4}),(x_{3},x_{7}),(x_{6},x_{12})$

       Обратные рёбра: $(x_{4},x_{3}),(x_{7},x_{6})$

       $\delta^* = min\{20-11,2-1,33-11\} = 1$

       $\varphi^* = min\{2,2\} = 2$

       $\varepsilon^* = min\{1, 2\} = 1$

       $\varphi=55$

       ![](https://www.plantuml.com/plantuml/png/bLNHReCm37pdAopwqchgYXC2eQsg_idK3RgXCPXe9lNpvr3gBdaOuOsUptTdErAF5tFgpSzN4pNLsO3feA_VFmoKTL-zcRfhetFNcQg7E8wMAuZtpSbKryFr1M7vjDflRZzZE5tWYh3uhjlFNLEtrRQlok7Z-RLhoXsHZ8LS3U7s2xSgORbEFGivroAbcEY9f7D5bDklt6f6ENFF8fjhaLDEwOdaSqMqsw_TQk5M2--UYNVCnAkZLqUlBcY2p7yH8c04UZVhzO7E3uj_QB250ZPGC5rEqINH9TDJGYZE1aJ2cB8i92p5YEp3P1Nn5TDnCZKcr43R9EL749kZO2WBz1pf-HHz63CYfp1rJ6t2b1kqnQob817r9tLciPmnVLyfAiPeMWBjVAWZiWwxh1NEc9wF6qB94DcCNfB0_0OMzvnIC4RMex-zimvTc7qhGdA3LHwMWWmDWoMtnHRtvpCXguckcKwffWvXBH_yyAT57w4jE8w5wCrb7m23qph3tKLqhDgI_Zr-0G==)
    5. Удалось пометить: $(x_{1}(+),x_{5}(+1),x_{8}(+5),x_{4}(-8),x_{3}(-4),x_{2}(-3),x_{6}(-2),x_{12}(+6))$

       Прямые рёбра: $(x_{1},x_{5}),(x_{5},x_{8}),(x_{6},x_{12})$

       Обратные рёбра: $(x_{8},x_{4}),(x_{4},x_{3}),(x_{3},x_{2}),(x_{2},x_{6})$

       $\delta^* = min\{41-31,15-12,33-12\} = 3$

       $\varphi^* = min\{15,1,5,6\} = 1$

       $\varepsilon^* = min\{3, 1\} = 1$

       $\varphi=56$

       ![](https://www.plantuml.com/plantuml/png/bPNTReCm38NlUGgBtRIgkkA4GD9MLTzawWRjq1XCD9Fw-BFvoKOk1jpHa-FFZdEadk_sqjZlpp8eywi5MqDJtDujP4MJlzcYhe9hNTkyWJ0CLci8Z-N5ve_JumLXjLaVTztF42vtU20yVHNLnw4igdpVv5dxyVnQbzc1J9QbCKF0QYizY5WAaGI9FOXS2ea9eZr8l1IYQ2NeGTHII4AJ20-IB8Mau_fqVAffJkMrq4jRc76zyT8HUT68l7Fqpj4xfsXi4ZzVSx91ATnkJssE_h717Wpim3YxdB9BiajdZqcXSNOW8gSfTh4MOq3jU-mUOfAKKt1oC8oeW1P6nQuNIy8uAPbYgAvdRq-dxErD0keqZuvy9OnTWHuqGH3rPqC9QwdJMXyl4fNJwBOa8eiweDRzutLNI9oIiejDqaae1fmaCUr4mngNIk4qelSj-BNqt9qmMKXNOFgdfAWEKZjxrEtE326Om6goQsUdgQPspIr__VFJGnzpIx2_5g9tBtvZ9qvpE4OeWdDUPVIFyWC=)
    6. Удалось пометить: $(x_{1}(+),x_{4}(+1),x_{8}(+4),x_{9}(+8),x_{7}(-9),x_{3}(-7),x_{2}(-3),x_{6}(-2),x_{12}(+6))$

       Прямые рёбра: $(x_{1},x_{4}),(x_{4},x_{8}),(x_{8},x_{9}),(x_{6},x_{12})$

       Обратные рёбра: $(x_{9},x_{7}),(x_{7},x_{3}),(x_{3},x_{2}),(x_{2},x_{6})$

       $\delta^* = min\{20-12,22-14,11-4,33-13\} = 7$

       $\varphi^* = min\{15,2,4,5\} = 2$

       $\varepsilon^* = min\{7, 2\} = 2$

       $\varphi=58$

       ![](https://www.plantuml.com/plantuml/png/bPNRJiCm38Rl-nHMkTc4nkgaXsIRfhq96hI3YjAYBaXxVEmU3Ci5RU-wBty-nucbdQxktBhlpogeYei3rq1RlhqxoCksU7LbKmUNfd552s4OhDOG7ggpAsx7spF2wd5zsFO_GpXVuORmy5NM7_kgh8jTM-JTnzDBK-Lx2Zb6zmu5guts96gfH9CayoHwgIIcRj2Jn4iboVt-v7urfNCghqIwj4H64kj9igKIGyt6diGibLYI64zYbqemyYOeyjRHMqTldIReCVN7SCu8qR1kDiTktFw9mGuiRC5AN4_5DSMrn6CY61DIaR24KufOZ06L7xGxO9G8mSc3OKGRwC9eizHYfAoWTCgHz3M7U3OQ0fBD4tJDOImR3BEK7IH9_bn2np9XNOxl2XDXr111P6W2AZsyJt-dgH0-PHVIWf1JoCreWbaFxLX2As5a7qhmWvXHED6aOelIie7Dt0zDuuYwimztPyTtdj0QYXk9qwccxffB_kRd3mXzpDs2Grk8Nb_ynWAdFJpCg89JKUVqt_83)
    7. Удалось пометить: $(x_{1}(+),x_{5}(+1),x_{9}(-5),x_{7}(-9),x_{6}(-7),x_{12}(+6))$

       Прямые рёбра: $(x_{1},x_{5}),(x_{6},x_{12})$

       Обратные рёбра: $(x_{5},x_{9}),(x_{9},x_{7}),(x_{7},x_{6})$

       $\delta^* = min\{41-32,33-15\} = 9$

       $\varphi^* = min\{8,13,1\} = 1$

       $\varepsilon^* = min\{9, 1\} = 1$

       $\varphi=59$

       ![](https://www.plantuml.com/plantuml/png/bPJTReCm38NlUGgBthIQkkA406chgcyoTODkQ0mccabz_Ddyf4ik1jpHpyVdE4xKyyrSMlFpLGLLSJLW6cZBzmy3UTaMRwPiwk3QDAPe8Go3zGR2OtKnnVrqVq5OFssEk_vd29SRt15MtsNzUQZAkjYtHTvzFBysLNuWaM7aUWYtNxXLwLQbMutTQknM5Ks7tdXgwNW9hBUfPv8iDKd9H7icwLAJpEtFtAgc2EL5wAKH67cN6NbrzEherGMjFF5VX9Wn1hsRxVP4rm-hVsIm1mqxq5Okf-IIvDBAOo8O4nAHPOfLB8ineFZ1TW-2aB849mV3Y1feOTHiinW9RL4ovQ7wp46UZWu1cSqpw0w7iMt8c2NiGIRgpn8wbbhMwNW9g2oZ0n54Xbb0qSFzz3jDB1cdH6aHYZ524anxg9dnKgMmZDo72BwGR3ISEAJW0OIq3UpCOVGQHzJDFknEF-uv9P9PGi5JwsJNXtx-6w2FkIquJ8xeZSxNQE6q3wyMHN0kwfp-oNy1)
---
## Минимальный разрез (?):
![](https://www.plantuml.com/plantuml/png/bPNRReCm343V-GgBldIQkjO90T9MLVzawWRTq1XCD9Fw-REvk2EQHdcfw8bpx3X672_krBZlpp8eyxC3LqDJlBqxo8ecVtL5NGNdkdPv0s4OB1yWt9SdbryFrsU4vUF3Vjtz3U5qWIl2ugkeFdPbKUNR9i_QXwUNkins5EGOZHrgl5-DL_LuLOzN8ygFNW7Ht08CB5UH9p5p9V5uVpnUJIY5zL8aSrEaT5ZZITAv4akIr9FOkHBS_64XE5V3uvFucja3XDu4eGfe2bUh0mqJBFxHm1OihC5Ak9uArnIk9JmYWX4XjH5cE8h38WmeVQ_TWY9aXE1aOTJDDVq4j5boCL9MK3pbC5tEFZmP3079xXEqXyD8DgJCOdQGnFngGiiIOMqSDm6DC3eGGMIO1fIwlv-kfx6GeKhKWb0DAJJ1f8EM6JTLAs5axrFmXQI3SEAGYWjGMZQmcPCX2cfhxtjdR_US44c5yE1Ej9Ejj_ZxPu0Uxjs2VUM8NkbyZGAdFTnQUf6EUPNHT-47)
---
Достигнут полный поток за 8 итераций.
Нашли 7 увеличивающих маршрута.
...
Подтверждается теорема Форда-Фалкерсона...

$\varphi_{max} = 59$
![](https://www.plantuml.com/plantuml/png/ZPJTRi8m38NlynHHTxDTCEgaVm64U9E9hO5LwgXKCgcFFxi_PivDkOk-Onyx9uNJFPpxyFlJgjPVWWwTxflhTz1rq_klq7GtTUcwu7kT9EhrJIU7zXpyS1m-uB2TZea-t_K0KZLIDLArKhLIpQIQIJMNQYxLGgg5L4kfbbAjf5f9rKdLHMcaKHnffCTnHNb151WODO3UR8wOdNvfcvlVzRxUq-7zislhFPO6GdgddTvgn-LshTnYkUNo30baY4pABASgAij0uVZPTgSDefm9h2u6AJPW4jZCimWPnwXOyyYdcNDvkJWeD7lEOBmSPDnG4Il80qtoVo6Ch6Gsrb48a3F32o44WfN2qVFxJ9aMJ9OjmJ82iuom2Ci9Ec8KgZNCq7qUGGzIBOOhbpIqWB7SGCuq33-p1OsxpzcvnpkNI2ecE7WjJd9zr3__3U3XsHQODmU8LgTdPBZkGz7Y3-daRpN-mVq1)
